In [8]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from torchinfo import summary
import torchvision.models as models

In [9]:
file = pd.read_csv('./data/encoded_data_v2.csv')
file.tail()

C:\Users\kdp\AppData\Local\Temp\ipykernel_11352\188211148.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv('./data/encoded_data_v2.csv')


,Unnamed: 0,image_name,comment_number,comment,0,1,2,3,4,5,...,68,69,70,71,72,73,74,75,76,77
158910,158910,998845445.jpg,0,A man in shorts and a hawaiian shirt leans ove...,0,2,0,89,0,0,...,1,1,1,1,1,1,1,1,1,1
158911,158911,998845445.jpg,1,A young man hanging over the side of a boat w...,0,10,2,276,0,0,...,1,1,1,1,1,1,1,1,1,1
158912,158912,998845445.jpg,2,A man is leaning off of the side of a blue and...,0,2,0,314,0,0,...,1,1,1,1,1,1,1,1,1,1
158913,158913,998845445.jpg,3,A man riding a small boat in a harbor with fo...,0,2,52,0,45,139,...,1,1,1,1,1,1,1,1,1,1
158914,158914,998845445.jpg,4,A man on a moored blue and white boat with hil...,0,2,0,0,6294,11,...,1,1,1,1,1,1,1,1,1,1


In [10]:
data = []
for k in range(file.shape[0]):
    sample = []
    img_dir = './data/flickr30k_images/'
    img_file = img_dir + file.iloc[k][1] 
    sample.append(img_file)

    capt = []
    for idx in range(78):
        capt.append(file.iloc[k][idx+4])
        # print(file.iloc[k][idx+4])

    sample.append(capt)
    data.append(sample)

In [45]:
for k in range(3):
    print(data[k])
    print(len(data[k][1]))
    print()

['./data/flickr30k_images/1000092795.jpg', [37, 10, 288, 0, 2075, 81, 147, 0, 0, 121, 0, 276, 0, 0, 0, 438, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
78

['./data/flickr30k_images/1000092795.jpg', [37, 10, 7, 662, 0, 33, 55, 273, 1379, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
78

['./data/flickr30k_images/1000092795.jpg', [37, 18, 0, 28, 209, 0, 16, 0, 0, 438, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
78



In [85]:
# 사용자 정의 데이터셋 정의
class CustomDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # 이미지와 캡션을 로드하고 전처리하여 반환
        image_path, caption = self.data[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        caption = torch.LongTensor(caption)
        return image, caption

In [86]:
# 데이터셋 인스턴스 생성
transform = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
dataset = CustomDataset(data, transform=transform)
print(len(dataset))

158915


In [87]:
for img, capt in dataset:
    print(img)
    print(type(img))
    print(capt)
    print(type(capt))
    break

tensor([[[-2.0665, -2.0323, -2.0323,  ...,  0.8276,  1.2214,  1.4269],
         [-2.0323, -2.0323, -1.9809,  ...,  0.4851, -0.3712, -0.1486],
         [-1.9638, -2.0323, -2.0323,  ..., -0.7137, -0.5767, -0.3369],
         ...,
         [-0.7308, -0.7479, -0.4054,  ...,  0.1939,  0.1939,  0.1939],
         [-0.4568, -0.1828, -0.0801,  ..., -0.0287,  0.2624,  0.3138],
         [ 0.4851,  0.2282,  0.2624,  ...,  0.2796,  0.4166,  0.3481]],

        [[-1.9482, -1.9307, -1.8957,  ...,  1.6408,  1.9559,  2.1835],
         [-1.9132, -1.9307, -1.8431,  ...,  1.3256,  0.6254,  0.9580],
         [-1.7906, -1.8957, -1.8957,  ...,  0.2577,  0.4853,  0.7479],
         ...,
         [-0.3550, -0.3550,  0.0651,  ...,  0.9755,  0.9930,  0.9930],
         [ 0.0476,  0.2052,  0.4678,  ...,  0.9405,  1.1155,  1.0805],
         [ 1.0105,  1.0455,  0.8880,  ...,  1.0280,  1.0980,  1.0630]],

        [[-1.7173, -1.7173, -1.6999,  ...,  1.6465,  2.1694,  2.3611],
         [-1.7173, -1.7173, -1.6476,  ...,  1

In [88]:
# dataset에서 train, valid, test를 나누기 
seed_gen = torch.Generator().manual_seed(42)
tr, val, ts = 0.7,0.1,0.2
trainDS, validDS, testDS = random_split(dataset, [tr, val, ts], generator=seed_gen)
print(len(trainDS), len(validDS), len(testDS))

111241 15891 31783


In [89]:
# dataloader 생성
batch_size = 256
train_dl = DataLoader(trainDS, batch_size=batch_size, shuffle=True, drop_last = True)
valid_dl = DataLoader(validDS, batch_size=batch_size, shuffle=True, drop_last = True)
test_dl = DataLoader(testDS, batch_size = batch_size, shuffle=True, drop_last = True)
print(len(train_dl), len(valid_dl), len(test_dl))

434 62 124


In [90]:
for img, capt in train_dl:
    print(capt)
    print()
    break

tensor([[   2,    0,    7,  ...,    1,    1,    1],
        [ 130,    2,    0,  ...,    1,    1,    1],
        [1443,  233,    0,  ...,    1,    1,    1],
        ...,
        [   0,   10,   15,  ...,    1,    1,    1],
        [   0,  222,  263,  ...,    1,    1,    1],
        [   0,  489,    0,  ...,    1,    1,    1]])



In [91]:
# 1개의 배치 안에 있는 이미지 확인
from torchvision.utils import make_grid

def show_batch(dl):
    """Plot images grid of single batch"""
    for (images, labels) in dl:
        fig,ax = plt.subplots(figsize = (16,12))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
        break
        
# show_batch(train_dl)

# 클래스 생성

### Image Captioning with Pytorch
- 필요한 모델 : CNN & RNN 
- 인코딩용 : CNN => Resnet
- 디코딩용 : RNN => LSTM
- CNN에서 나온 결과물을 LSTM에 연결 

In [100]:
# 인코딩용 CNN 모델 생성 : RESNET18 (가중치O)

resnet = models.resnet18()
# 전결합층 변경
resnet.fc = nn.Linear(in_features = 512, out_features = 1)

# 모델의 합성곱층 가중치 고정 (완전 연결층은 학습시켜야함)
for name, param in resnet.named_parameters():
    param.requires_grad = False
for name, param in resnet.fc.named_parameters():
    param.requires_grad = True 

In [120]:
# 디코딩용 RNN 모델 생성 : LSTM

from torch.nn.utils.rnn import pack_padded_sequence
# 패딩된 시퀀스를 실제 데이터 길이에 맞게 패킹하여 효율적인 연산을 수행할 수 있게 해줌

class decoder(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers = 1):
        super(decoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, features, captions, lengths):
        embedding = self.embed(captions)
        embedding = torch.cat((features.unsqueeze(2), embedding.unsqueeze(1)),1)
        packed = pack_padded_sequence(embedding, lengths, batch_first = True)
        hiddens, _ = self.lstm(packed)
        outputs = self.linear(hiddens[0])
        return outputs

- class로 만든거

In [94]:
# 인코딩용 CNN 모델 생성 : RESNET18 (가중치O)

# res_model = models.resnet18(weights = ( "ResNet18_Weights.DEFAULT"))
# 전결합층 변경
# res_model.fc = nn.Linear(in_features = 512, out_features = 1)


# class encoderCNN(nn.Module):
#     def __init__(self, embed_size):
#         super(encoderCNN, self).__init_()
#         self.resnet = models.resnet18(weights = ( "ResNet18_Weights.DEFAULT"))
#         # 전이학습 모델의 전결합층 변경
#         self.resent.fc = nn.Linear(self.resenet.fc.in_features, embed_size)

#         self.dropout = nn.Dropout(0.5)  # 흠,, 필요할까?
#         self.relu = nn.Relu()
    
#     def forward(self, x):
#         features = self.resnet(x)
        
#         # 모델의 합성곱층 가중치 고정 (완전 연결층은 학습시켜야함)
#         for name, param in self.resnet.named_parameters():
#             param.requires_grad = False
#         for name, param in self.resnet.fc.named_parameters():
#             param.requires_grad = True 

#         result = self.relu(features)

#         return result

In [111]:
# 디코딩용 RNN 모델 생성 : LSTM

# class decoderRNN(nn.Module):
#     def __init__ (self, embed_size, vocab_size, hidden_size, num_layers):
#         super(decoderRNN, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_size)
#         self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
#         self.linear = nn.Linear(hidden_size, vocab_size)
#         self.dropout = nn.Dropout(0.5) # 흠 필요함?
    
#     def forward(self, features, caption):
#         # embeddings = self.dropout(self.embedding(caption))   # 왜 dropout?
#         embeddings = self.embedding(caption)
#         embeddings = torch.cat((features.unsqueeze(0), embeddings), dim = 0)
#         hiddens, _ = self.lstm(embeddings)
#         outputs = self.linear(hiddens)
#         return outputs

In [112]:
# CNN 과 RNN을 연결시키자

# class CNN2RNN(nn.Module):
#     def __init__(self, embed_size, vocab_size, hidden_size, num_layers):
#         super(CNN2RNN, self).__init__()
#         self.encoderCNN = encoderCNN(embed_size)
#         self.decoderRNN = decoderRNN(embed_size, vocab_size, hidden_size, num_layers)
    
#     def captionImage(self, image, vocabulary, maxlength = 50):
#         result_caption = []

#         with torch.no_grad():
#             x = self.encoderCNN(image).unsqueeze(0)
#             states = None

#             for _ in range(maxlength) :
#                 hiddens, states = self.decoderRNN.lstm(x, states)
#                 output = self.decoderRNN.linear(hiddens.squeeze(0))
#                 predicted = output.argmax(1)
#                 print(predicted.shape)

#                 result_caption.append(predicted.item())
#                 x = self.decoderRNN.embedding(output).unsqueeze(0)

#                 if vocabulary.itos[predicted.item()] == "<EOS>" :
#                     break
        
#         return [vocabulary.itos[i] for i in result_caption]

# 학습을 위한 하이퍼파라미터 정의

In [116]:
embed_size = 256
hidden_size = 512
vocab_size = 21652  # 단어사전 크기
num_layers = 1
model = decoder(embed_size, hidden_size, 21653, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [108]:
for image, caption in train_dl:
    print(caption[0])
    a = np.array(caption[0])

    print(len(np.nonzero(a)[0]))
    num_list = []
    for ind in range(len(image)):
        a = np.array(caption[ind])
        num_list.append(len(np.nonzero(a)[0]))
    print(num_list)
    break

torch.Tensor(num_list), len(num_list)

tensor([ 0, 17,  0, 48,  0,  0, 69,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1])
74
[74, 73, 73, 73, 71, 73, 72, 70, 74, 71, 73, 68, 73, 73, 73, 73, 69, 76, 72, 76, 75, 72, 75, 71, 66, 75, 71, 71, 69, 73, 74, 72, 71, 69, 71, 72, 72, 73, 68, 77, 76, 74, 73, 68, 75, 72, 73, 75, 77, 72, 74, 72, 73, 73, 70, 72, 75, 73, 76, 76, 73, 64, 68, 77, 73, 71, 73, 75, 75, 73, 73, 71, 76, 77, 75, 72, 73, 75, 75, 73, 70, 67, 71, 72, 74, 74, 73, 75, 71, 78, 72, 70, 74, 76, 76, 73, 74, 74, 71, 73, 75, 74, 73, 71, 73, 72, 70, 73, 71, 72, 73, 75, 77, 76, 73, 72, 70, 74, 62, 71, 72, 76, 73, 72, 76, 76, 72, 70, 70, 74, 75, 75, 75, 66, 71, 77, 70, 77, 74, 71, 76, 73, 70, 72, 73, 77, 70, 70, 71, 75, 75, 73, 73, 70, 76, 74, 73, 69, 74, 75, 74,

(tensor([74., 73., 73., 73., 71., 73., 72., 70., 74., 71., 73., 68., 73., 73.,
         73., 73., 69., 76., 72., 76., 75., 72., 75., 71., 66., 75., 71., 71.,
         69., 73., 74., 72., 71., 69., 71., 72., 72., 73., 68., 77., 76., 74.,
         73., 68., 75., 72., 73., 75., 77., 72., 74., 72., 73., 73., 70., 72.,
         75., 73., 76., 76., 73., 64., 68., 77., 73., 71., 73., 75., 75., 73.,
         73., 71., 76., 77., 75., 72., 73., 75., 75., 73., 70., 67., 71., 72.,
         74., 74., 73., 75., 71., 78., 72., 70., 74., 76., 76., 73., 74., 74.,
         71., 73., 75., 74., 73., 71., 73., 72., 70., 73., 71., 72., 73., 75.,
         77., 76., 73., 72., 70., 74., 62., 71., 72., 76., 73., 72., 76., 76.,
         72., 70., 70., 74., 75., 75., 75., 66., 71., 77., 70., 77., 74., 71.,
         76., 73., 70., 72., 73., 77., 70., 70., 71., 75., 75., 73., 73., 70.,
         76., 74., 73., 69., 74., 75., 74., 73., 75., 77., 75., 75., 72., 72.,
         70., 72., 75., 73., 76., 75., 75., 74., 72.

# 학습, 검증, 테스트 함수 정의

In [121]:
import torchmetrics.functional as metrics

def training(dataloader):
    model.train()
    loss_list = []
    acc_list = []
    precision_list = []
    recall_list = []
    f1score_list = []

    for images, captions in dataloader:        
        num_list = []
        for ind in range(len(image)):
            a = np.array(caption[ind])
            num_list.append(len(np.nonzero(a)[0]))
    
        lengths = torch.Tensor(num_list)     #근데 이게 배치사이즈가 맞나??
        
        targets = pack_padded_sequence(captions, lengths, batch_first = True, enforce_sorted=False)[0]

        # 이미지 특성 추출
        features = resnet(images)
        features = features.view(features.size(0), -1)

        # 예측
        outputs = model(features, captions, lengths)
        print(outputs)
        # 역전파
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 정확도
        acc = metrics.accuracy(outputs, label, task = 'multiclass') 
        precision = metrics.precision(outputs, label, task = 'multiclass')
        recall = metrics.recall(outputs, label, task = 'multiclass')
        f1score = metrics.f1_score(outputs, label, task = 'multiclass')
        
        loss_list.append(loss)
        acc_list.append(acc)
        precision_list.append(precision)
        recall_list.append(recall)
        f1score_list.append(f1score)
        
    total_loss = sum(loss_list) / len(loss_list)
    total_acc = sum(acc_list) / len(acc_list)
    total_precision = sum(precision_list) / len(precision_list)
    total_recall = sum(recall_list)/len(recall_list)
    total_f1score = sum(f1score_list) / len(f1score_list)
    # print(f"[TOTAL Train Loss] ==> {total_loss}")
    # print(f"ACC : {total_acc}  Precision : {total_precision}  Recall : {total_recall} F1score : {total_f1score}")
    return total_loss, total_acc, total_precision, total_recall, total_f1score

training(train_dl)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 256 for tensor number 1 in the list.